In [ ]:
!pip install opencv-python tf-models-official "tensorflow-text==2.9.*"

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow_models import nlp
from keras import metrics, layers, optimizers, losses, Model
from keras.utils import plot_model
import json

In [ ]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

In [ ]:
!mkdir -p squad
!curl -o squad/train-v2.0.json https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!curl -o squad/dev-v2.0.json https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
!python3 models/official/nlp/data/create_finetuning_data.py \
 --squad_data_file=/content/squad/train-v2.0.json \
 --vocab_file=gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12/vocab.txt \
 --train_data_output_path=/content/squad/train_v2.0.tf_record \
 --meta_data_file_path=/content/squad/squad_v2.0_meta_data \
 --fine_tuning_task_type=squad \
 --max_seq_length=384 \
 --version_2_with_negative=True

In [ ]:
database_dir = "squad"
bert_dir = database_dir+"/bert_base"
squad_version = "v2.0"
squad_dir = database_dir+"/squad"
max_seq_length = 384
train_batch_size = 10

raw_dataset = tf.data.TFRecordDataset([os.path.join(
    squad_dir, f"train_{squad_version}.tf_record")])